In [1]:
using Pkg 
Pkg.instantiate()
using QAlgebra

using ComplexRationals
using LaTeXStrings

Precompiling project...
   1168.5 ms  ✓ QAlgebra
  1 dependency successfully precompiled in 2 seconds. 7 already precompiled.


In [2]:
using Test
CF_TYPES = (CAtom, CExp, CLog, CRational, CProd, CSum)
VARS = ["x", "y"]

function gen_examples(::Type{T}, depth::Int, max_depth::Int) where T<:CFunction
    # -- one small collection of truly atomic CAtom examples:
    atomic = [
        CAtom(2,      [0,0]),
        CAtom(2,      [1,0]),
        CAtom(3//2,   [0,1]),
        CAtom(crationalize(1+2im), [1,1])
    ]
    # convenience:
    a = atomic[1]

    # at max depth, return exactly one “simple” example per type
    if depth >= max_depth
        if T === CAtom
            return [a]
        elseif T === CSum
            return [CSum([a, a])]
        elseif T === CProd
            return [CProd(a, a)]
        elseif T === CRational
            return [CRational(a, a)]
        elseif T === CExp
            return [CExp(a)]
        elseif T === CLog
            return [CLog(a)]
        else
            error("Unhandled type $T")
        end
    end

    # otherwise depth < max_depth: build composites
    if T === CAtom
        return atomic
    elseif T === CSum
        ex = CFunction[]
        for U in CF_TYPES, V in CF_TYPES
            e1 = gen_examples(U, depth+1, max_depth)[1]
            e2 = gen_examples(V, depth+1, max_depth)[1]
            push!(ex, CSum([e1,e2]))
        end
        return ex
    elseif T === CProd
        ex = CFunction[]
        for U in CF_TYPES, V in CF_TYPES
            e1 = gen_examples(U, depth+1, max_depth)[1]
            e2 = gen_examples(V, depth+1, max_depth)[1]
            push!(ex, CProd(e1, e2))
        end
        return ex
    elseif T === CRational
        ex = CFunction[]
        for U in CF_TYPES, V in CF_TYPES
            e1 = gen_examples(U, depth+1, max_depth)[1]
            e2 = gen_examples(V, depth+1, max_depth)[1]
            push!(ex, CRational(e1, e2))
        end
        return ex
    elseif T === CExp
        ex = CFunction[]
        for U in CF_TYPES
            e1 = gen_examples(U, depth+1, max_depth)[1]
            push!(ex, CExp(e1))
        end
        return ex
    elseif T === CLog
        ex = CFunction[]
        for U in CF_TYPES
            e1 = gen_examples(U, depth+1, max_depth)[1]
            push!(ex, CLog(e1))
        end
        return ex
    else
        error("Unhandled type $T")
    end
end


in_vsc() = any(startswith(key, "VSCODE_") for key in keys(ENV))
function line(msg::String=""; tail::String="=", heads::Vector{String}=[">", "<"], n::Int=-1)
    if n < 0
        if !in_vsc()
            n, _ = displaysize(stdout)
        else 
            n = 70
        end
        if n == 0
            n = 70
        end
    end
    m = length(msg)
    if m == 0
        println(tail^n)
        return
    end
    n2 = Int(floor((n - m - 4)/2)) 
    if n2 < 0
        n2 = 0
    end
    final_str = join([tail^n2, heads[1], " ", msg, " ", heads[2], tail^n2], "")
    if length(final_str) < n 
        final_str = final_str * tail 
    end
    println(final_str) 
    return
end

macro test_succeeds(expr, msg_func="")
    quote
        @test begin
            try
                $(esc(expr))
                true
            catch err
                line($(esc(msg_func)))
                line(string(typeof(err)))
                Base.showerror(stdout, err)
                line()
                println()
                rethrow(err)
                false
            end
        end
    end
end

list_of_subtypes = subtypes(CFunction)
for T in list_of_subtypes
    @assert T in list_of_subtypes "Type $T not included in test -> add to test!"
end



max_depth = 1
# collect by type
examples_by_type = Dict{DataType, Vector{CFunction}}()
for T in CF_TYPES
    examples_by_type[T] = gen_examples(T, 0, max_depth)
end
println("Finished creating examples of all types." )

# flatten
all_ex = reduce(vcat, values(examples_by_type))

# a fixed small var‐name list for to_string


# string generation
for ex in all_ex
    @test_succeeds to_string(ex, VARS)   "to_string($ex) failed"
    @test_succeeds to_string(ex, VARS, do_latex=true)   "to_string($ex, VARS, do_latex=true) failed"
end
println("Finished creating strings and LaTeXStrings of all examples. ")

Finished creating examples of all types.
Finished creating strings and LaTeXStrings of all examples. 


In [4]:
# binary arithmetic on every pair
for ex1 in all_ex, ex2 in all_ex
    @test_succeeds ex1 + ex2  "$ex1 + $ex2 failed"
    @test_succeeds ex1 - ex2  "$ex1 - $ex2 failed"
    @test_succeeds ex1 * ex2  "$ex1 * $ex2 failed"
    @test_succeeds ex1 / ex2  "$ex1 / $ex2 failed"
end

# simplify, sorting, recursive_sort!
for ex1 in all_ex, ex2 in all_ex
    @test_succeeds QAlgebra.CFunctions.simplify(ex1)            "simplify($ex1) failed"         
end


In [7]:
#a = CAtom(1, [1,0])
#b = CAtom(2, [1,2])
a = CAtom(ComplexRational(1,0,1), [1,0])
b = CAtom(ComplexRational(2,0,1), [1,2])
s = QAlgebra.CFunctions.simplify(a/(a+b))
s = to_string(a/(-a*b-a), ["\\alpha", "\\beta"], do_latex=true, do_frac=true)
latexstring(s)

UndefVarError: UndefVarError: `CAtom` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [8]:
s = to_string(a/b, ["\\alpha", "\\beta"], do_latex=true, do_frac=true)
latexstring(s)

UndefVarError: UndefVarError: `a` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [10]:
QAlgebra.CFunctions.simplify((a+1)*(b/a))*exp(a)

UndefVarError: UndefVarError: `QAlgebra` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: QAlgebra is loaded but not imported in the active module Main.

In [11]:
pre, s = to_stringer(QAlgebra.CFunctions.simplify(exp(b)/(-a*b-a)), ["\\alpha", "\\beta"], do_latex=true, do_frac=true)
latexstring(s)

UndefVarError: UndefVarError: `QAlgebra` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: QAlgebra is loaded but not imported in the active module Main.

In [12]:
my_sign, my_str = to_stringer(-2*a*b-4*b, ["\\alpha", "\\beta(t)"], do_latex=false, braced=true)
latexstring(my_str) 

UndefVarError: UndefVarError: `a` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [13]:
my_sign, my_str = to_stringer(-(1+2im)*b*a^(-2), ["\\alpha", "\\beta(t)"], do_latex=true, braced=true, do_frac=true)
latexstring(my_str)

UndefVarError: UndefVarError: `b` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [16]:
function simple_combinable_F(t1::Union{CAtom, CSum}, t2::Union{CAtom, CSum})::Tuple{Bool, ComplexRational}
    # Transform CAtom's to CSums 
    if t1 isa CAtom
        t1 = CSum([t1])
    end
    if t2 isa CAtom
        t2 = CSum([t2])
    end
    @assert length(t1) == length(t2) "Cannot separate pair terms with different lengths."

    # check if corresponding terms have the similar exponents (i.e. the difference of exponents has to be the same for each term in CSum)
    exponents_diff = a1.terms[1].var_exponents .- a2.terms[1].var_exponents
    if !all([diff == 0 for diff in exponents_diff])
        return false, ComplexRational(0, 0, 0)
    end
    for (a1, a2) in zip(t1.terms[2:end], t2.terms[2:end])
        if a1.var_exponents .- a2.var_exponents != exponents_diff
            return false, ComplexRational(0, 0, 0)
        end
    end
    
    # check if the terms have a constant ratio in the coefficients 
    ratio = t1.terms[1].coeff / t2.terms[1].coeff
    if !(ratio.b == 0 || ratio.a == 0)
        return false, ratio
    end
    for (a1, a2) in zip(t1.terms[2:end], t2.terms[2:end])
        if a1.coeff / a2.coeff != ratio
            return false, ratio
        end
    end
    return true, ratio
end
# check for multiple elements in a Vector of CSum 
function simple_combinable_Fs(ts::Vector{Union{CAtom,CSum}})::Tuple{Vector{Vector{Union{CAtom, CSum}}}, Vector{Vector{Int}}}
    groups = Vector{Vector{Union{CAtom,CSum}}}()
    indexes = Vector{Vector{Int}}()
    for (i, t) in enumerate(ts)
        placed = false
        for (inds, grp) in zip(indexes, groups)
            ok,_ = simple_combinable_F(grp[1], t)
            if ok 
                push!(grp, t)
                push!(inds, i)
                placed = true
                break
            end
        end
        if !placed
            push!(groups, [t])
            push!(indexes, [i])
        end
    end
    return groups, indexes
end

function ratios_Fs(ts::Vector{Union{CAtom,CSum}})::Vector{ComplexRational}
    if ts[1] isa CAtom 
        c1 = ts[1].coeff
    else 
        c1 = ts[1].terms[1].coeff
    end
    ratios = ComplexRational[ComplexRational(1,0,1)]
    for t in ts[2:end] 
        if t isa CAtom 
            c2 = t.coeff
        else 
            c2 = t.terms[1].coeff
        end
        push!(ratios, c1/c2)
    end
    return ratios
end
function group_Fs(ts::Vector{Union{CAtom,CSum}})::Union{CAtom,CSum, Tuple{CAtom, Vector{Union{CAtom, CSum}}}}
    # find the correct way to group a group of Fs (they must be groupable, create the input vector with simple_combinable_Fs)
    if length(ts) == 1
        return ts[1]
    elseif length(ts) > 1
        # we need to find the best common ratio 
        ratios = ratios_Fs(ts)
        base, multiples = common_denominator_form(ratios)
        pre_F = ts[1]*base 
        post_Fs = [ t*m for (t,m) in zip(ts, multiples)]
        return (pre_F, post_Fs)
    end
end

"""
    how_to_combine_Fs(ts::Vector{Union{CAtom, CSum}}) :: Tuple{Vector{Union{CAtom, CSum, Tuple{CAtom, Vector{Union{CAtom, CSum}}}}}, Vector{Vector{Int}}}

Groups and combines `CAtom` and `CSum` objects in the input vector `ts` into composite structures that can be processed together. 
Returns a tuple containing both the groups as (CFunction) elements of a Vector and the indexs corresponding to the elements in the groups. 
The (CFunction) grouping is given either by a single CFunction element (either a single `CAtom` or a `CSum`) or by a tuple of an `CAtom` (F1) containing the shared factors, and a vector of CFunction elements (F2_i), so that 
together they represent a term of the form: F1 * (F2_1 + ... + F2_n). 
"""
function how_to_combine_Fs(ts::Vector{Union{CAtom,CSum}})::Tuple{Vector{Union{CAtom,CSum, Tuple{CAtom, Vector{Union{CAtom, CSum}}}}}, Vector{Vector{Int}}}
    if length(ts) == 1
        return [ts[1]], [[1]]
    elseif length(ts) > 1 
        groups, indexes = simple_combinable_Fs(ts)
        return [group_Fs(grp) for grp in groups], indexes
    else
        return [], []
    end
end

UndefVarError: UndefVarError: `CAtom` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [17]:
function QComposite2string(q::QAtomProduct; do_latex::Bool=true, do_sigma::Bool=false, braced::Bool=true, do_frac::Bool=true)::Tuple{Bool, String}
    num = is_numeric(q)
    if is_numeric
        curr_sign, curr_str = to_stringer(q.coeff_fun, variable_str_vec(q, do_latex=do_latex), braced=braced, do_frac=do_frac)
        return curr_sign, curr_str
    else
        curr_sign, curr_str = to_stringer(q.coeff_fun, variable_str_vec(q, do_latex=do_latex), braced=braced, do_frac=do_frac)
        operator_str = join([qAtom2string(t, q.statespace, do_latex=do_latex, do_sigma=do_sigma) for t in q.expr], "")
        connector = do_latex ? raw"\cdot" : "*"
        if printnumeric(q.coeff_fun)
            return curr_sign, curr_str * connector * operator_str
        else
            return curr_sign, operator_str 
        end
    end
end

function QComposites2string(terms::Vector{QComposite}; do_latex::Bool=false, do_sigma::Bool=false, braced::Bool=true, do_frac::Bool=true, separate_sign::Bool=false)::String
    substrings::Vector{{Tuple{Bool, String}} = [QComposite2string(t, do_latex=do_latex, do_sigma=do_sigma, braced=braced, do_frac=do_frac) for t in terms]
    # connect substrings  
    if separate_sign
        string = substrings[1][2]
    else
        string = substrings[1][1] ? "-" * substrings[1][2] : substrings[1][2]
    end
    for (curr_sign, curr_str) in substrings[2:end]
        string *= curr_sign ? "-" * curr_str : "+" * curr_str
    end
    if separate_sign 
        return substrings[1][1], string
    end
    return string
end
import .CFunction: how_to_combine_Fs
function group_qAtomProducts(qs::Vector{QAtomProduct})::Vector{Union{QAtomProduct, Tuple{Union{CAtom, CSum}, Vector{QAtomProduct}}}
    coeffs_funs = [q.coeff_fun for q in qs]
    coeff_groups, indexes = how_to_combine_Fs(coeffs_funs)
    new_qs = []
    for (coeffs, indexes) in zip(coeff_groups, indexes)
        if !( coeff isa Tuple )
            push!(new_qs, copy(qs[indexes[1]]))
        else
            pre_F = coeffs[1]
            post_F = coeffs[2]
            post_q = QAtomProduct[]
            for (F, i) in zip(post_F, indexes) 
                new_p = copy(qs[i]) 
                new_p.coeff_fun *= F 
                push!(post_q, new_p) 
            end 
            push!(new_qs, (pre_F, post_q)) 
        end
    end 
    return new_qs 
end 

function qAtomProduct_group2string(qs::QAtomProduct; do_latex::Bool=true, do_sigma::Bool=false, braced::Bool=true, do_frac::Bool=true)::String
    curr_sign, operator_str = QComposite2string(qs[1], do_latex=do_latex, do_sigma=do_sigma, braced=braced, do_frac=do_frac)
    return curr_sign, "", operator_str
end
function qAtomProduct_group2string(qs::Tuple{Union{CAtom, CSum}, Vector{QAtomProduct}; do_latex::Bool=true, do_sigma::Bool=false, braced::Bool=true, do_frac::Bool=true)::String
    # assume the qs can be simple grouped (see the functions: simple_combinable_Fs, group_Fs)
    F = qs[1]
    qs = qs[2]
    f_sign, f_str = to_stringer(F; do_latex=do_latex, braced=braced, do_frac=do_frac)
    q_str = QComposites2string(qs; do_latex=do_latex, do_sigma=do_sigma, braced=braced, do_frac=do_frac, separate_sign=false)  # don't worry about internal signs, this has already been taken care off by the sign handling of the grouping 
    return f_sign, f_str, q_str
end 

function allnegative(x::Tuple{Bool, String})::Bool 
    return x[1]
end
function allnegative(x::Vector{Tuple{Bool, String}})::Bool
    return all(allnegative, x)
end
function brace(x::String; do_latex::Bool=true)::String
    if do_latex 
        return raw"\left(" * x * raw"\right)"
    else
        return "(" * x * ")"
    end
end

function QExpr2string(q::QExpr; do_latex::Bool=true, do_sigma::Bool=false, braced::Bool=true, do_frac::Bool=true, do_grouped::Bool=true, return_grouping::Bool=false)::Union{Tuple{Bool, String}, Tuple{Bool, String, Bool}}
    # outputs sign, string, {optional return_grouping:} single_group::Bool   => return grouping implies that the expression will be braced if it isn't already! , hence the outputted sign is handled differently 
    # first sort terms 
    q_sorted = sort(q)
    if !do_grouped # outside (not inside of a QComposite)
        if !return_grouping
            return QComposites2string(q_sorted.terms, do_latex=do_latex, do_sigma=do_sigma, braced=braced, do_frac=do_frac, separate_sign=true)
        else
            return QComposites2string(q_sorted.terms, do_latex=do_latex, do_sigma=do_sigma, braced=braced, do_frac=do_frac, separate_sign=true), false
    else  # iunside amnother QComposite
        # separate into QAtomProduct and other QComposite terms -> sorting puts qAtomProducts first 
        # then group qAtomProducts by their factors 
        first_non_qAtomProduct = findfirst(x -> !isa(x, QAtomProduct), q_sorted.terms)
        if first_non_qAtomProduct === nothing
            first_non_qAtomProduct = length(q_sorted.terms)
        end
        qAtomProduct_terms = q_sorted.terms[1:first_non_qAtomProduct]
        other_terms = q_sorted.terms[first_non_qAtomProduct+1:end]
        groups = group_qAtomProducts(qAtomProduct_terms)
        # create strings for each element 
        all_strings = []
        for group in groups
            curr_sign, first, second = qAtomProduct_group2string(group, do_latex=do_latex, do_sigma=do_sigma, braced=braced, do_frac=do_frac)
            push!(all_strings, (curr_sign, first*brace(second, do_latex=do_latex))
        end
        for term in other_terms
            push!(all_strings, QComposite2string(term, do_latex=do_latex, do_sigma=do_sigma, braced=braced, do_frac=do_frac))
        end
        
        # make QComposites2string better, by allowing a third output in case of single group. to traverse multiple layers of Composites within composites. 
        if return_grouping 
            # do we switch the sign? 
            if allnegative(all_strings) || ( get_default(:FLIP_IF_FIRST_TERM_NEGATIVE ) && all_strings[1][1] )
                # switch signs 
                first_sign = true 
                all_strings = [(!sign, s) for (sign, s) in all_strings]
            else
                first_sign = false
            end
        else
            first_sign = all_strings[1][1]
        end
        total_string = all_strings[1][1] ? "-" * all_strings[1][2] : all_strings[1][2]
        for (sign, s) in all_strings[2:end]
            if sign 
                total_string *= "-" * s
            else 
                total_string *= "+" * s
            end
        end
        if return_grouping 
            single_group::Bool = length(groups) == 1 && length(all_strings) == 1
            return first_sign, total_string, single_group
        else
            return first_sign, total_string
        end
    end
end



UndefVarError: UndefVarError: `QAtomProduct` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [18]:
sort([((1,2),[1,2]), ((3,4), [12,2])])

2-element Vector{Tuple{Tuple{Int64, Int64}, Vector{Int64}}}:
 ((1, 2), [1, 2])
 ((3, 4), [12, 2])

In [22]:
const DEFAULT_COEFF_PREFS = Dict(
    :FLOAT_DIGITS => 2,
    :EXP_DIGITS => 2,
    :FLIP_IF_FIRST_TERM_NEGATIVE  => true,
    )

function get_default(name::Symbol)
    return DEFAULT_COEFF_PREFS[name]
end

get_default (generic function with 1 method)

In [ ]:
using ComplexRationals
abstract type CFunction end

mutable struct CAtom      <: CFunction
    coeff::ComplexRational
    var_exponents::Vector{Int}
    function CAtom(coeff::Int, var_exponents::Vector{Int})
        c = ComplexRational(coeff, 0, 1)
        return new(c, var_exponents)
    end
    function CAtom(coeff::Rational, var_exponents::Vector{Int})
        c = ComplexRational(numerator(coeff), 0, denominator(coeff))
        return new(c, var_exponents)
    end
    function CAtom(coeff::ComplexRational, var_exponents::Vector{Int})
        return new(coeff, var_exponents)
    end
end

mutable struct CSum       <: CFunction
    terms::Vector{CFunction}
    # (inner) constructor for a Vector{<:CFunction>, with flattening
    function CSum(ts::AbstractVector{<:CFunction})
        flat = CFunction[]
        for t in ts
            if t isa CSum
                append!(flat, (t::CSum).terms)   # flatten nested sums
            else
                push!(flat, t)
            end
        end
        new(flat)
    end
end

# varargs “outer” constructor so you can call CSum(a,b,c) directly
CSum(ts::CFunction...) = CSum(collect(ts))

mutable struct CRational  <: CFunction
    numer::CSum
    denom::CSum
end

_terms(f::CFunction) = f isa CSum ? (f::CSum).terms : [f]

import Base: isless
function isless(a::CAtom, b::CAtom)
    a.var_exponents < b.var_exponents    # Vector{Int} has lex order
end


import Base: iszero, isempty

iszero(a::CAtom)        = iszero(a.coeff)
iszero(s::CSum)         = isempty(s.terms) || all(iszero, s.terms)
iszero(r::CRational)    = iszero(r.numer)

isempty(s::CSum)        = isempty(s.terms)

# true if no variables (all exponents zero)
isnumeric(a::CAtom)    = all(e->e==0, a.var_exponents)
isnumeric(s::CSum)     = all(isnumeric, s.terms)
isnumeric(r::CRational)= isnumeric(r.numer) && isnumeric(r.denom)

allnegative(a::CAtom) = is_negative(a.coeff)
allnegative(s::CSum)  = !isempty(s.terms) && all(allnegative, s.terms)
allnegative(r::CRational) = allnegative(r.numer)

min_exponents(a::CAtom)    = a.var_exponents
function min_exponents(s::CSum)
    if length(s.terms) == 0
        return []
    end
    min_vals = min_exponents(s.terms[1])
    for term in s.terms[2:end]
        curr_min_vals = min_exponents(term)
        min_vals = min.(min_vals, curr_min_vals)
    end
    return min_vals
end
function min_exponents(r::CRational)
    min_vals = min_exponents(r.numer)
    min_vals2 = min_exponents(r.denom)
    return min.(min_vals, min_vals2)
end


import Base: length, getindex, iterate, deleteat!, reverse
function length(p::CSum)::Int
    return length(p.terms)
end
getindex(p::CSum, i::Int) = p.terms[i]
iterate(p::CSum, state=1) = state > length(p.terms) ? nothing : (p.terms[state], state + 1)
deleteat!(p::CSum, i::Int) = CSum(deleteat!(p.terms, i))
reverse(q::CSum) = CSum(reverse(q.terms))
dims(q::CAtom) = length(q.var_exponents)
dims(q::CSum) = dims(q.terms[1])
dims(q::CRational) = dims(q.numer) 

import Base: +, -, *, /, ^, ==

# addition always builds a flat sum
+(a::CFunction) =  a 
+(a::CFunction, b::CFunction) = CSum( vcat(_terms(a), _terms(b)) )

# unary minus & subtraction
import Base: -, +

-(a::CAtom) = CAtom(-a.coeff, a.var_exponents)
-(s::CSum) = CSum([ -t for t in s.terms ])
-(r::CRational) = CRational(-r.numer, r.denom)
-(a::CFunction, b::CFunction) = a + (-b)


# distribute * over sums
*(A::CSum, B::CSum)       = CSum([ x*y for x in A.terms,   y in B.terms ])
*(A::CSum, b::CFunction)  = CSum([ x*b for x in A.terms ])
*(a::CFunction, B::CSum)  = CSum([ a*y for y in B.terms ])

# atom‐level ×
*(a::CAtom, b::CAtom)     = CAtom(crationalize(a.coeff*b.coeff), a.var_exponents .+ b.var_exponents)
*(a::CAtom, r::CRational) = CRational(CSum(a)*r.numer, r.denom)
*(r::CRational, a::CAtom) = CRational(r.numer*CSum(a), r.denom)
*(a::CRational, b::CRational) = CRational(a.numer*b.numer, a.denom*b.denom)
*(a::CSum, b::CRational) = CRational(a*b.numer, b.denom)
*(b::CRational, a::CSum) = CRational(a*b.numer, b.denom)
# number 
*(a::CFunction, b::Number)  = a*CAtom(b, zeros(Int, dims(a)))
*(b::Number, a::CFunction)  = CAtom(b, zeros(Int, dims(a))) * a
multiply_one(a::CRational, b::Int) = (a.numer * b) / (a.denom * b)

# division
/(A::CSum, B::CSum) = length(B.terms)==1 ? CSum([ x/B.terms[1] for x in A.terms ]) : CRational(A, B)
/(A::CAtom, B::CSum)     = CRational(CSum(A), B)
/(A::CSum, b::CAtom)     = CSum([ x/b for x in A.terms ])
/(a::CAtom, b::CAtom)     = CAtom(a.coeff/b.coeff, a.var_exponents .- b.var_exponents)
/(a::CAtom, r::CRational) = CRational(CSum(a)*r.denom, r.numer)
/(r::CRational, a::CAtom) = CRational(r.numer, r.denom*CSum(a))
/(a::CRational, b::CRational) = CRational(a.numer*b.denom, a.denom*b.numer)
/(a::CRational, b::CSum) = CRational(a.numer, b.numer*b)
/(a::CSum, b::CRational) = CRational(a*b.denom, b.numer)
/(a::CFunction, n::Number) = a/CAtom(n, zeros(Int, dims(a)))
/(n::Number, a::CFunction) = CAtom(n, zeros(Int, dims(a))) / a

# exponentiation 
^(A::CSum, n::Int) = CSum([ x^n for x in A.terms ])
^(A::CAtom, n::Int) = CAtom(A.coeff^n, A.var_exponents .* n)
^(a::CRational, n::Int) = CRational(a.numer^n, a.denom^n)

function ==(a::CAtom, b::CAtom)
    return (a.coeff == b.coeff && a.var_exponents == b.var_exponents)
end
function ==(a::CSum, b::CSum)
    if length(a) != length(b)
        return false
    end
    for i in 1:length(a) 
        if a[i] != b[i]
            return false
        end
    end
    return true
end
function ==(a::CRational, b::CRational)
    return (a.numer == b.numer && a.denom == b.denom)
end

function addable(a::CAtom, b::CAtom)::Bool
    return a.var_exponents == b.var_exponents
end
function addable(a::CRational, b::CRational)::Bool
    return a.denom == b.denom
end
function addable(a::CSum, b::CSum)::Bool
    true
end
# assume inifiable
function unify_add(a::CAtom, b::CAtom)::CFunction
    absum = a.coeff+b.coeff
    if iszero(absum)
        var_zeros = zeros(Int, dims(a))
        return CAtom(0, var_zeros)
    end
    return CAtom(absum, a.var_exponents)
end
function unify_add(a::CRational, b::CRational)::CFunction
    simple_numer = simplify(a.numer+b.numer)
    if iszero(simple_numer)
        var_zeros = zeros(Int, dims) 
        return CAtom(0, var_zeros)
    end
    return CRational(simple_numer, a.denom)
end
function unify_add(a::CSum, b::CSum)::CFunction
    return simplify(a+b)
end

unify (generic function with 3 methods)

In [24]:
import Base: sort!, sort

function complexrational2coeffs(a::ComplexRational)::Vector{Int}
    return [a.a, a.b, a.c]
end

function sort_key(a::CAtom)
    return vcat(a.var_exponents, complexrational2coeffs(a.coeff))
end

function sort_key(s::CSum)
    keys = [sort_key(t) for t in s.terms]
    return vcat(length(keys), keys...)
end
function sort_key(r::CRational)
    dkeys = sort_key(r.denom)
    nkeys = sort_key(r.numer)
    return vcat(length(dkeys), dkeys, nkeys)
end

function sort!(s::CSum; by=sort_key)
    sort!(s.terms, by=by)
end
function sort!(s::CRational, by=sort_key)
    sort!(s.numer.terms, by=by)
    sort!(s.denom.terms, by=by)
end
function sort!(s::CAtom; by=sort_key)
    # do nothing 
    return s
end

function sort(s::CSum; by=sort_key)::CSum
    return CSum(sort(s.terms, by=by))
end
function sort(s::CRational; by=sort_key)::CRational
    numer = sort(s.numer.terms, by=by)
    denom = sort(s.denom.terms, by=by)
    return CRational(numer, denom)
end
function sort(s::CAtom; by=sort_key)::CAtom
    # do nothing 
    return s
end

# Test 
a = CAtom(2, [0,1])
b = CAtom(3, [1,0])
c = CAtom(4, [1,1])
d = CSum([a,c,b,c])
sort(d)

CSum(CFunction[CAtom(2, [0, 1]), CAtom(3, [1, 0]), CAtom(4, [1, 1]), CAtom(4, [1, 1])])

In [25]:
issimple(f::CFunction) = error("Not implemented for type $(typeof(f)).")
issimple(f::CSum) = all(t -> t isa CAtom, f.terms)
issimple(f::CRational) = issimple(f.numer) && issimple(f.denom)
issimple(f::CAtom) = true


issimple (generic function with 4 methods)

In [26]:
# divisors 
function divisors(a::CFunction)
    error("Not implemented for type $(typeof(a)).")
end
function divisors(a::CAtom)::Vector{Int}
    return [a.coeff.c]
end
function divisors(a::CSum)::Vector{Int}
    return reduce(vcat, [divisors(t) for t in a.terms])
end
function divisors(a::CRational)::Vector{Int}
    return reduce(vcat, [divisors(t) for t in [a.numer, a.denom]])
end
# simplify rationals by multiplying the 

divisors (generic function with 4 methods)

In [27]:
function vec_multiply(x::CAtom, vector::Vector{Int})::CAtom
    return CAtom(x.coeff, x.var_exponents - vector)
end
function vec_multiply(x::CSum, vector::Vector{Int})::CSum
    return CSum([vec_multiply(t, vector) for t in x.terms])
end
function vec_multiply(x::CRational, vector::Vector{Int})::CRational
    return CRational(vec_multiply(x.numer), vec_multiply(x.denom))
end

vec_multiply (generic function with 3 methods)

In [28]:
# Assumes that the divisors are 1 
function coeffs(s::CAtom)::Vector{Int}
    # if subtype complex take real and imaginary parts separately 
    return [s.coeff.a, s.coeff.b]
end
function coeffs(s::CSum)::Vector{Int}
    return vcat([coeffs(t) for t in s.terms]...)
end
function coeffs(s::CRational)::Vector{Int}
    return vcat(coeffs(s.numer), coeffs(s.denom))
end
import Base: gcd
function gcd(s::CFunction)
    return gcd(coeffs(s))   
end
function gcd(s::CFunction, t::CFunction)
    return gcd(vcat(coeffs(s), coeffs(t)))
end
#gcd(a*2+b+a*4)

gcd (generic function with 19 methods)

In [ ]:
function simplify(s::CAtom)
    return s 
end
function simplify(r::CRational)
    n = simplify(r.numer)
    d = simplify(r.denom)
    # remove fractions in coefficients in Rational
    curr_div = vcat(divisors(n), divisors(d))
    factor = lcm(curr_div...)
    n = factor * n
    d = factor * d
    # common denominator
    factor = gcd(n, d)
    n = n / factor
    d = d / factor

    min_n = min_exponents(n)
    min_d = min_exponents(d)
    min_vals = min.(min_n, min_d)
    n = vec_multiply(n, min_vals)
    d = vec_multiply(d, min_vals)
    if allnegative(d) || FLIP_IF_FIRST_TERM_NEGATIVE && allnegative(d[1]))   # prefer negatives on numerator
        n = -n
        d = -d
    end
    # if denom now has exactly one term, collapse back to a sum
    if length(d.terms) == 0
        error("Divinding by zero")
    elseif length(d.terms) == 1
        # use our /‐overload to divide each term in n by d.terms[1]
        return n / d.terms[1]
    else
        return CRational(n, d)
    end
end

function simplify(s::CSum)
    # first simplify the lower levels 
    elements = [simplify(e) for e in s.terms]
    s = CSum(elements)
    sort!(s)
    i = 1
    elements = s.terms
    new_elements = CFunction[]
    curr_element = elements[1]
    for i in 2:length(elements)
        if typeof(curr_element) == typeof(elements[i]) 
            if addable(curr_element, elements[i])
                curr_element = unify_add(curr_element, elements[i])
            else
                if !iszero(curr_element)
                    push!(new_elements, curr_element)
                end
                curr_element = elements[i]
            end
        else
            if !iszero(curr_element)
                push!(new_elements, curr_element)
            end
            curr_element = elements[i]
        end
    end
    if !iszero(curr_element)
        push!(new_elements, curr_element)
    end
    if length(new_elements) == 0 
        push!(new_elements, CAtom(0, zeros(Int, dims(s))))
    end
    return CSum(new_elements)
end


"""
    max_exponents(f::CFunction) -> Vector{Int}

For an CAtom: its own exponents.  
For an CSum: elementwise max over all terms.  
For an CRational: returns a pair `(max_numer, max_denom)`.
"""
max_exponents(a::CAtom) = abs.(a.var_exponents)
function max_exponents(s::CSum)
    if length(s.terms) == 0
        return []
    else
        max_m = max_exponents(s[1])
        for t in s.terms[2:end]
            curr_max = max_exponents(t)
            for i in eachindex(curr_max)
                if curr_max[i] > max_m[i]
                    max_m[i] = curr_max[i] 
                end
            end
        end
        return max_m
    end
end
function max_exponents(r::CRational)
    max_a = max_exponents(r.numer)
    max_b = max_exponents(r.denom)
    return [max(a, b) for (a, b) in zip(max_a, max_b)]
end
"""
    build_xpows(x::Vector{<:Number}, M::Vector{Int})

Returns a vector xpows of length `length(x)`, where
  xpows[j][k+1] = x[j]^k,  for k = 0:M[j].
"""
function build_xpows(x::Vector{<:Number}, max_exp::Vector{Int})::Vector{Vector}
    @assert length(x) == length(M)
    xpows = [Vector{typeof(x[i])}(undef, length(x)) for i in eachindex(x)]
    for j in eachindex(x)
        xpows[j] = [ x[j]^k for k in 0:max_exp[j] ]
    end
    return xpows
end


function evaluate(a::CAtom, x::AbstractVector{<:Number})
    a.coeff * prod(x[i]^a.var_exponents[i] for i in eachindex(a.var_exponents))
end
function evaluate(s::CSum, x::AbstractVector{<:Number})
    sum(evaluate(t, x) for t in s.terms)
end
function evaluate(r::CRational, x::AbstractVector{<:Number})
    evaluate(r.numer, x) / evaluate(r.denom, x)
end

# Evaluate but with xpows
function evaluate(a::CAtom, xpows::Vector{Vector})
    return a.coeff * prod(xpows[i][a.var_exponents[i]+1] for i in eachindex(a.var_exponents))
end
function evaluate(s::CSum, xpows::Vector{Vector})
    return sum(evaluate(t, xpows) for t in s.terms)
end
function evaluate(r::CRational, xpows::Vector{Vector})
    return evaluate(r.numer, xpows) / evaluate(r.denom, xpows)
end

Base.Meta.ParseError: ParseError:
# Error @ /home/micha/Documents/PhD/Research/Projects/QAlgebra.jl/prototyping/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X40sZmlsZQ==.jl:22:74
    d = vec_multiply(d, min_vals)
    if allnegative(d) || FLIP_IF_FIRST_TERM_NEGATIVE && allnegative(d[1]))   # prefer negatives on numerator
#                                                                        ╙ ── unexpected `)`

In [30]:
function sign_string(c::ComplexRational, do_latex::Bool=false)::Tuple{Bool, String}
    if is_negative(c)
        return (true, string(c, do_latex=do_latex)[2:end])
    else
        return (false, string(c, do_latex=do_latex))
    end
end
is_abs_one(c::ComplexRational)::Bool = (abs(c.a) == abs(c.b))
function is_abs_one(c::CFunction)
    if isnumeric(c)
        if isa(c, CAtom)
            return is_abs_one(c.coeff)
        elseif isa(c, CSum)
            if length(c) == 1
                return is_abs_one(c[1])
            else
                return false
            end
        else
            return false
        end
        return true
    else
        return false
    end
end

is_abs_one (generic function with 2 methods)

In [31]:
# --- Generic string constructor ---
function stringer(f::CFunction)::Tuple{String,String}
    error("No fallback method for CFunction type: $(typeof(f))")
end

# --- CAtom ---
function stringer(a::CAtom)::Tuple{Bool,String}  # true = minus, false = plus
    if isnumeric(a)
        return sign_string(a.coeff) 
    else
        c = a.coeff
        vec = string(a.var_exponents)
        sign, c_str = sign_string(c) 
        if is_abs_one(c)
            return (sign, vec)
        else
            return sign,  c_str*"*"*vec
        end
    end
end

# --- CSum ---
function stringer(s::CSum; braced::Bool=false) ::Tuple{Bool,String}
    # braced specifies whether the terms will be grouped, so that an external sign is needed
    s = simplify(s)
    terms = s.terms
    if braced 
        if allnegative(s) || FLIP_IF_FIRST_TERM_NEGATIVE && allnegative(s[1]))
            # if all negative and braced, we can just negate the whole thing
            sig = true
            _, body = stringer(-s)
            return sig, body
        else
            sig = false
            body = join(stringer(t) for t in terms)
        end
    end

    # process each term into (sign, body)
    parts = String[]
    for (i, t) in enumerate(terms)
        sig, body = stringer(t)
        if i == 1
            # first term keeps its sign, but no space if positive
            push!(parts, sig == true ? "-$body" : body)
        else
            push!(parts, sig == true ? "-$body" : "+$body")
        end
    end
    return false, join(parts, "")
end


# --- CRational ---
function stringer(r::CRational)::Tuple{Bool,String}
    r = simplify(r)
    n = r.numer
    d = r.denom

    # check first term in numerator
    n_sig, n_body = stringer(n, braced=true)
    d_sig, d_body = stringer(d)  # we ignore sign of denom

    if length(n) > 1
        n_body = "($n_body)"
    end
    if length(d) > 1
        d_body = "($d_body)"
    end

    return n_sig, "$n_body/$d_body"
end

# --- Show ---
import Base: show

function show(io::IO, f::CFunction)
    sig, body = stringer(f)
    sig_str = sig ? "-" : ""
    print(io, sig_str * body)
end



a = CAtom(1, [1,0])
b = CAtom(2, [0,1])
p = CSum([a, b])
s = simplify(a/(a*b+a))
#evaluate(s, [2,3])
#max_exponents(s)

Base.Meta.ParseError: ParseError:
# Error @ /home/micha/Documents/PhD/Research/Projects/QAlgebra.jl/prototyping/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X42sZmlsZQ==.jl:28:78
    if braced 
        if allnegative(s) || FLIP_IF_FIRST_TERM_NEGATIVE && allnegative(s[1]))
#                                                                            ╙ ── unexpected `)`

In [32]:
const superscript_indexes = Dict('a' => "ᵃ", 'b' => "ᵇ", 'c' => "ᶜ", 'd' => "ᵈ", 'e' => "ᵉ", 'f' => "ᶠ",
    'g' => "ᵍ", 'h' => "ʰ", 'i' => "ⁱ", 'j' => "ʲ", 'k' => "ᵏ", 'l' => "ˡ", 'm' => "ᵐ", 'n' => "ⁿ",
    'o' => "ᵒ", 'p' => "ᵖ", 'q' => "ᵠ", 'r' => "ʳ", 's' => "ˢ", 't' => "ᵗ", 'u' => "ᵘ", 'v' => "ᵛ",
    'w' => "ʷ", 'x' => "ˣ", 'y' => "ʸ", 'z' => "ᶻ", '2' => "²", '3' => "³", '4' => "⁴", '5' => "⁵", '6' => "⁶", '7' => "⁷", '8' => "⁸", '9' => "⁹", '1' => "", '-' => "⁻", '=' => "⁼")

function str2sup(s::String)::String
    new_str = ""
    for c in s
        if haskey(superscript_indexes, Char(c))
            new_str *= superscript_indexes[Char(c)]
        else
            #@warn "Character $c not found in superscript_indexes, printing as ^$c instead. Avoid this by choosing one of the following characters: $superscript_indexes.keys()"
            new_str *= "^$c"
        end
    end
    return new_str
end

str2sup (generic function with 1 method)

In [33]:
# Generic fallback
function stringer(f::CFunction, vars::Vector{String}; do_latex::Bool=false)
    error("No stringer method for type $(typeof(f)) with variable names")
end

# --- CAtom with variable names ---
function stringer(a::CAtom, vars::Vector{String}; do_latex::Bool=false)
    exps = a.var_exponents
    @assert length(vars) == length(exps) "Number of symbols must match number of variables"

    if isnumeric(a)
        return sign_string(a.coeff) 
    else
        # build the variable part
        varparts = String[]
        for (i, e) in enumerate(exps)
            if e == 0
                continue
            elseif do_latex
                push!(varparts, e == 1 ? vars[i] : "$(vars[i])^{$e}")
            else
                push!(varparts, e == 1 ? vars[i] : "$(vars[i])"*str2sup(string(e)))
            end
        end

        var_str = isempty(varparts) ? "" : join(varparts, do_latex ? "\\cdot " : "")
        c = a.coeff
        sign, c_str = sign_string(c) 
        if is_abs_one(c)
            return (sign, vec)
        else
            return sign,  c_str*"*"*var_str
        end
    end
end

# --- CSum with variable names ---
function stringer(s::CSum, vars::Vector{String}; do_latex::Bool=false, braced::Bool=false)
    s = simplify(s)
    terms = s.terms
    if isempty(terms)
        return false, "0"
    end

    parts = String[]
    for (i, t) in enumerate(terms)
        sig, body = stringer(t, vars; do_latex=do_latex)
        if i == 1
            push!(parts, sig ? "-" * body : body)
        else
            push!(parts, sig ? "-" * body : "+" * body)
        end
    end

    out = join(parts, "")
    return false, out
end

# --- CRational with variable names ---
function stringer(r::CRational, vars::Vector{String}; do_latex::Bool=false)
    r = simplify(r)
    n = r.numer
    d = r.denom

    n_sig, n_str = stringer(n, vars; do_latex=do_latex, braced=true)
    _, d_str = stringer(d, vars; do_latex=do_latex, braced=true)

    if do_latex
        return n_sig, "\\frac{$n_str}{$d_str}"
    else
        n_wrapped = length(n.terms) > 1 ? "($n_str)" : n_str
        d_wrapped = length(d.terms) > 1 ? "($d_str)" : d_str
        return n_sig, "$n_wrapped/$d_wrapped"
    end
end

"""
    to_string(f::CFunction, vars::Vector{String};
              do_latex::Bool = false,
              braced::Bool = false) -> String

Converts the CFunction `f` into a string using variable names from `vars`.

If `do_latex=true`, uses LaTeX syntax (e.g., `\\frac{}` and `x^2`).  
If `braced=true`, wraps the expression in `()` or `{}` if it's a sum.

The result includes the correct sign (`-`) if the top-level expression is negative.
"""
function to_stringer(f::CFunction, vars::Vector{String}; do_latex::Bool=false, braced::Bool=false)::Tuple{Bool, String}
    if braced && f isa CSum && length(f) > 1
        sig, body = stringer(f, vars; do_latex=do_latex, braced=braced)
    else
        sig, body = stringer(f, vars; do_latex=do_latex)
    end
    # Apply braces to sums if requested
    if braced && f isa CSum && length(f) > 1
        body = do_latex ? "\\left( $body \\right)" : "($body)"
    end
    return sig, body
end
function to_string(f::CFunction, vars::Vector{String}; do_latex::Bool=false, braced::Bool=false, optional_sign::Bool=true)::String
    sig, body = to_stringer(f, vars; do_latex=do_latex, braced=braced)

    if sig
        return "-" * body
    else
        if optional_sign
            return body
        end
        return "+" *body
    end
end

#a = CAtom(1, [1,0])
#b = CAtom(2, [1,2])
a = CAtom(ComplexRational(1,0,1), [1,0])
b = CAtom(ComplexRational(2,0,1), [1,2])
s = simplify(a/(a+b))
s = to_string(a/(a*b+a), ["x", "y"], do_latex=false)

MethodError: MethodError: no method matching simplify(::CRational)
The function `simplify` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  simplify(!Matched::CAtom)
   @ Main ~/Documents/PhD/Research/Projects/QAlgebra.jl/prototyping/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X40sZmlsZQ==.jl:1


In [34]:
s = to_string(a/(a*b+a), ["\\alpha", "\\beta"], do_latex=true)
using LaTeXStrings
latexstring(s)

MethodError: MethodError: no method matching simplify(::CRational)
The function `simplify` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  simplify(!Matched::CAtom)
   @ Main ~/Documents/PhD/Research/Projects/QAlgebra.jl/prototyping/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X40sZmlsZQ==.jl:1
